# CARLA Python API - Lab5
## Radar Emergency Braking

This solution covers:
- radar sensor setup,
- TTC computation from radar detections,
- automatic emergency braking under TTC threshold,
- test scenario and cleanup.

Docs:
- Radar sensor: https://carla.readthedocs.io/en/latest/ref_sensors/#radar-sensor
- VehicleControl: https://carla.readthedocs.io/en/latest/python_api/#carlavehiclecontrol
- Python API: https://carla.readthedocs.io/en/latest/python_api/


In [ ]:
import carla, time, math, random


In [ ]:
client = carla.Client("localhost", 2000)
client.set_timeout(10.0)
world = client.get_world()
spectator = world.get_spectator()


## Helper functions


In [ ]:
def move_spectator_to(transform, spectator, distance=8.5, z=3.2, pitch=-14.0):
    back = transform.location - transform.get_forward_vector() * distance
    loc = carla.Location(back.x, back.y, back.z + z)
    rot = carla.Rotation(pitch=pitch, yaw=transform.rotation.yaw, roll=0.0)
    spectator.set_transform(carla.Transform(loc, rot))


def safe_destroy(actors):
    for a in actors:
        if a is not None:
            try:
                a.destroy()
            except RuntimeError:
                pass


def spawn_vehicle(world, spawn_index=0, vehicle_filter="vehicle.tesla.model3"):
    points = world.get_map().get_spawn_points()
    if not points:
        raise RuntimeError("No spawn points found")
    bps = world.get_blueprint_library().filter(vehicle_filter)
    if not bps:
        bps = world.get_blueprint_library().filter("vehicle.*")
    for k in range(len(points)):
        actor = world.try_spawn_actor(random.choice(bps), points[(spawn_index + k) % len(points)])
        if actor is not None:
            return actor
    raise RuntimeError("Could not spawn vehicle")


def spawn_target_ahead(world, ego, distances=(22.0, 28.0, 34.0)):
    wp = world.get_map().get_waypoint(ego.get_transform().location, project_to_road=True, lane_type=carla.LaneType.Driving)
    bps = world.get_blueprint_library().filter("vehicle.*")
    for d in distances:
        cands = wp.next(float(d))
        random.shuffle(cands)
        for c in cands:
            actor = world.try_spawn_actor(random.choice(bps), c.transform)
            if actor is not None:
                return actor
    raise RuntimeError("Could not spawn target vehicle ahead")


def spawn_radar(world, attach_to, transform=None, h_fov=25.0, v_fov=8.0, r=50.0):
    bp = world.get_blueprint_library().find("sensor.other.radar")
    bp.set_attribute("horizontal_fov", str(h_fov))
    bp.set_attribute("vertical_fov", str(v_fov))
    bp.set_attribute("range", str(r))
    bp.set_attribute("points_per_second", "1500")
    if transform is None:
        transform = carla.Transform(carla.Location(x=2.2, z=1.0), carla.Rotation(pitch=4.0))
    return world.spawn_actor(bp, transform, attach_to=attach_to)


def speed_kmh(actor):
    v = actor.get_velocity()
    return 3.6 * ((v.x * v.x + v.y * v.y + v.z * v.z) ** 0.5)


def compute_min_ttc(radar_measurement, az_deg=12.0, alt_deg=5.0, max_depth=55.0):
    az = math.radians(az_deg)
    alt = math.radians(alt_deg)
    best_ttc = None
    best_depth = None
    best_vrel = 0.0

    for d in radar_measurement:
        if abs(d.azimuth) > az or abs(d.altitude) > alt:
            continue
        if d.depth <= 0.3 or d.depth > max_depth:
            continue

        # abs handles sign convention differences across CARLA versions.
        vrel = abs(float(d.velocity))
        if vrel < 0.1:
            continue

        ttc = float(d.depth) / vrel
        if best_ttc is None or ttc < best_ttc:
            best_ttc = ttc
            best_depth = float(d.depth)
            best_vrel = vrel

    return best_ttc, best_depth, best_vrel


## Complete solution run


In [ ]:
original_settings = world.get_settings()
new_settings = world.get_settings()
new_settings.synchronous_mode = True
new_settings.fixed_delta_seconds = 0.05
world.apply_settings(new_settings)

ego = None
target = None
radar = None

state = {
    "min_ttc": None,
    "min_depth": None,
    "vrel": 0.0,
}

ttc_threshold = 1.6
cruise_throttle = 0.38
max_runtime = 35.0


def on_radar(data):
    ttc, depth, vrel = compute_min_ttc(data)
    state["min_ttc"] = ttc
    state["min_depth"] = depth
    state["vrel"] = vrel


try:
    ego = spawn_vehicle(world, spawn_index=4, vehicle_filter="vehicle.tesla.model3")
    target = spawn_target_ahead(world, ego, distances=(22.0, 28.0, 34.0))

    # keep target static to validate emergency braking clearly
    target.apply_control(carla.VehicleControl(throttle=0.0, brake=1.0, hand_brake=True))

    radar = spawn_radar(world, ego)
    radar.listen(on_radar)

    start = time.time()
    next_print = 0.0
    brake_activated = False

    while time.time() - start < max_runtime:
        world.tick()

        ttc = state["min_ttc"]
        depth = state["min_depth"]

        ctrl = carla.VehicleControl()
        if ttc is not None and ttc < ttc_threshold:
            ctrl.throttle = 0.0
            ctrl.brake = 1.0
            brake_activated = True
        else:
            ctrl.throttle = cruise_throttle
            ctrl.brake = 0.0

        ego.apply_control(ctrl)
        move_spectator_to(ego.get_transform(), spectator)

        if ttc is not None:
            txt = "TTC={:.2f}s depth={:.1f}m v_rel={:.1f}m/s".format(ttc, depth, state["vrel"])
            col = carla.Color(255, 60, 60) if ttc < ttc_threshold else carla.Color(60, 220, 60)
            world.debug.draw_string(ego.get_transform().location + carla.Location(z=2.2), txt, color=col, life_time=0.08)

        now = time.time()
        if now >= next_print:
            ttc_txt = "None" if ttc is None else "{:.2f}".format(ttc)
            depth_txt = "None" if depth is None else "{:.1f}".format(depth)
            print("speed={:5.1f} km/h | TTC={} s | depth={} m | brake={:.2f}".format(speed_kmh(ego), ttc_txt, depth_txt, ctrl.brake))
            next_print = now + 1.0

        if brake_activated and speed_kmh(ego) < 0.5:
            print("Emergency braking completed: ego vehicle stopped.")
            break

except KeyboardInterrupt:
    pass
finally:
    if radar is not None:
        radar.stop()
    safe_destroy([radar, target, ego])
    world.apply_settings(original_settings)


## Suggested experiments

1. Reduce `ttc_threshold` to `1.0` and compare stopping distance.
2. Increase target distance and confirm no false emergency brake.
3. Replace static target with moving target and observe TTC dynamics.
